# Thesis pictures
This notebooks contains the code used to produced most of the report images

In [3]:
%reload_ext autoreload
%autoreload 2

import cv2
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

from os import path
from utilities.patches import *
from utilities.postprocessing.handlers.functional import read_image

from jupyterthemes import jtplot
jtplot.reset()

plt.style.use(['science', 'ieee'])
# matplotlib.rcParams['figure.figsize'] = (5,4)
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""


In [2]:
def hmshow(hm, save_path=None):
    fig = plt.figure(dpi=800)
    plt.tight_layout()
    sns.heatmap(hm, cmap=plt.cm.viridis, vmin=0, vmax=1)
    plt.show()
    if save_path is not None: fig.savefig(save_path, bbox_inches='tight')
    return fig
def hm_name_from_path(hm):
    return path.basename(hm)

def figsave(fig, out_path, ax=None):
    if ax is not None:
        ax.view_init(elev=20, azim=125)

        ax.tick_params(axis = 'both', which = 'major', labelsize = 4, pad=0)
        ax.tick_params(axis = 'both', which = 'minor', labelsize = 4, pad=0)
#     plt.show()
    fig.savefig(out_path, bbox_inches='tight', transparent=True)

## Height maps pictures

## 2D

In [1]:
out_dir = '/home/francesco/Documents/Master-Thesis/papers/Thesis/img/hm/'
hms = glob.glob('./maps/new-train/*.png')
name2path = {hm_name_from_path(hm) : hm for hm in hms}

hmshow(hmread(hms[0]) / 255, '{}/{}'.format(out_dir, hm_name_from_path(hms[0])))
for hm in hms:
    hmshow(hmread(hm) / 255, '{}/{}'.format(out_dir, hm_name_from_path(hm)))
    


## 3D

In [9]:
out_dir = '/home/francesco/Documents/Master-Thesis/papers/Thesis/img/hm3d/'


In [9]:
from utilities.postprocessing.handlers.functional import read_image
out_dir = '/home/francesco/Documents/Master-Thesis/papers/Thesis/img/hm3d/'
for hm in hms:
    im = read_image(hm).astype(np.float32) / 255
    p = MyPatch.from_hm(im)
    p.plot3d_mayavi(0.02, '{}/{}'.format(out_dir, hm_name_from_path(hm)))
    

In [ ]:
# matplotlib.rcParams['figure.figsize'] = (5,4)
# matplotlib.rcParams['font.size'] = 6

for hm in hms:
    fig, ax = Patch.from_hm(hmread(hm) / 255).plot3d(title='', colorbar=True, rstride=1, cstride=1)
#     ax.set_aspect(0.5)
#     ax.set_zlim(0, 1)
    ax.view_init(elev=40, azim=-45)
    ax.tick_params(axis = 'both', which = 'major', labelsize = 4, pad=0)
    ax.tick_params(axis = 'both', which = 'minor', labelsize = 4, pad=0)
    plt.show()

    fig.savefig('{}/{}'.format(out_dir, hm_name_from_path(hm)),  bbox_inches = "tight", transparent=True)
    

## Quarry

In [10]:
hm = './maps/test/querry-big-10.png'
im = read_image(hm).astype(np.float32) / 255

p = Patch.from_hm(im * 10)
p.plot3d_mayavi(0.02, size=(2000, 2000), azimuth=45, elevation=45, distance=85, save_path='{}/{}'.format(out_dir, hm_name_from_path(hm)))


<mayavi.modules.surface.Surface object at 0x7fbb598d4888> None


## Patch pictures
### store the patch and map position for some simulations

In [ ]:
# %matplotlib notebook
matplotlib.rcParams['figure.figsize'] = (8,8)
fig = plt.figure()
height_factor = 3
fig, ax = Patch.from_hm((hmread(hm) / 255)).plot3d(title='', dpi=800,  colorbar=False)
ax.view_init(elev=20, azim=125)

ax.tick_params(axis = 'both', which = 'major', labelsize = 4, pad=0)
ax.tick_params(axis = 'both', which = 'minor', labelsize = 4, pad=0)
plt.show()
fig.savefig('{}/{}'.format(out_dir, hm_name_from_path(hm)), bbox_inches='tight', transparent=True)

In [ ]:
from utilities.visualisation import *
from utilities.postprocessing.utils import KrockPatchExtractStrategy

### from the wall map

In [ ]:
df_base_dir = '/media/francesco/saetta/krock-dataset/train/csvs_patches/'
meta = pd.read_csv('/media/francesco/saetta/krock-dataset/train/bags/meta.csv')
meta = meta[meta['map'] == 'bars1']
hm_dir = '/media/francesco/saetta/krock-dataset/train/patches/'

In [ ]:
df = pd.read_csv('{}/{}.csv'.format(df_base_dir, meta['filename'].iloc[0]))
# for some reason map name is missing
df['map_name'] = 'bars1'
hm = hmread('./maps/train/bars1.png')

In [ ]:
df_vis = PatchesAndDataframeVisualization(df,hm, patch_extractor=KrockPatchExtractStrategy)

In [ ]:
%matplotlib notebook
df_vis.show_patches_on_the_map(df=df_vis.df[0:4], compress=False)

# Data aug

In [12]:
out_dir = '/home/francesco/Documents/Master-Thesis/papers/Thesis/img/data-aug/'

## Center patch


In [30]:
from utilities.patches import *
from estimators.data.transformations import *
class SquareInTheMiddlePatch(Patch, Mayavi3dPlottable):
    def make(self, size=4, *args, **kwargs):
#         here we can access the .hm filled with zeros
        self.hm[self.shape[0] // 2 - size : self.shape[0] // 2 + size, self.shape[1] // 2 - size: self.shape[0] // 2 + size] = 1
        return self.hm
    
center_p = SquareInTheMiddlePatch((100,100))(size=25)

In [40]:
center_p = SquareInTheMiddlePatch((100,100))(size=25)

fig, ax = center_p.plot3d(rstride=1, cstride=1)

figsave(fig, out_dir +'/3d/square-middle.png', ax)

fig = center_p.plot2d()
figsave(fig, out_dir +'/2d/square-middle.png', None)


In [39]:
transform = CenterAndScalePatch()

center_p_tr = transform(center_p.hm)
center_p_tr = Patch.from_hm(center_p_tr)

fig, ax = center_p_tr.plot3d(rstride=1, cstride=1)

figsave(fig, out_dir +'/3d/square-middle-center.png', ax)

fig = center_p_tr.plot2d()
figsave(fig, out_dir +'/2d/square-middle-center.png', None)
# plt.show()

/home/francesco/.local/lib/python3.6/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


## Data-aug

### Simplex noise

In [45]:
random_simplex_noise = RandomSimplexNoise((100,100), n=5)

100%|██████████| 5/5 [00:00<00:00, 23.04it/s]


In [49]:
flat_path = Patch((513,513))

In [50]:
center_p_simplex = Patch.from_hm(im2simplex(flat_path.hm.copy(), feature_size=5, scale=8))

In [52]:
for i in range(1,5):
    center_p_simplex = MyPatch.from_hm(im2simplex(flat_path.hm.copy(), feature_size=10 * (i + 1), scale=8))
#     fig, ax = center_p_simplex.plot3d(rstride=1, cstride=1)
#     figsave(fig out_dir + '/3d/simplex{}.png'.format(i), ax)
    fig = center_p_simplex.plot3d_mayavi(0.02, out_dir + '/3d/simplex{}.png'.format(i))
#     fig = plt.gcf()
#     figsave(fig, out_dir + '/3d/simplex{}.png'.format(i), ax)


### Dropout

In [ ]:
center_p.plot2d()

In [43]:
# wall_p = WallPatch((100,100), size=8)()
wall_p = center_p
wall_p_dp = Patch.from_hm(iaa.Dropout(p=(0.05, 0.1)).augment_image(wall_p.hm))

fig = wall_p_dp.plot2d()
figsave(fig, out_dir + '/2d/center-dropout.png')

### Croase Dropout

In [ ]:
# wall_p = WallPatch((100,100), size=8)()
wall_p_dp = Patch.from_hm(iaa.CoarseDropout((0.02, 0.1),size_percent=(0.6, 0.8)).augment_image(wall_p.hm))

fig = wall_p_dp.plot2d()
plt.show()
figsave(fig, out_dir + '/2d/center-coarse-dropout.png')

In [ ]:
x = Patch.from_hm(im2simplex(center_p.hm.copy(), feature_size=25, scale=8))
fig = x.plot2d()
plt.show()
figsave(fig, out_dir + '/2d/center-simplex.png')

In [ ]:
x = RampPatch((100,100))()

fig = x.plot2d()
figsave(fig, out_dir + '/2d/ramp.png')

x = iaa.Dropout(p=(0.05, 0.1)).augment_image(x.hm)
x = iaa.CoarseDropout((0.02, 0.1),size_percent=(0.6, 0.8)).augment_image(x)
x = im2simplex(x.copy(), scale=8)

x = Patch.from_hm(x)

fig = x.plot2d()
# x.plot3d()

figsave(fig, out_dir + '/2d/ramp-aug.png')

## Spawn

In [ ]:
out_dir = '/home/francesco/Documents/Master-Thesis/papers/Thesis/img/3/spawn/'

In [ ]:
from simulation.env.spawn import FlatGroundSpawnStrategy

In [ ]:
spawn_strat = FlatGroundSpawnStrategy('./maps/train/bars1.png')

In [ ]:
flat_pos = spawn_strat.find_spawn_points(40)
fig = spawn_strat.show_spawn_pos(flat_pos)
figsave(fig, out_dir + '/flat-spawn-10.png')

In [ ]:
spawn_pos = spawn_strat.reduce_positions_by_clustering(flat_pos, 20)
fig = spawn_strat.show_spawn_pos(spawn_pos)
figsave(fig, out_dir + '/spawn-10.png')

In [2]:
import glob

In [10]:
def make_subfigure(img_path, make_caption, size):
     return "\begin{{subfigure}}[b]{{}\textwidth}\includegraphics[width=\linewidth]{{}}\end{{subfigure}}".format(size, img_path)


## Generate latex for quarry patches

In [36]:
import glob
ps = sorted(glob.glob('/home/francesco/Documents/Master-Thesis/papers/Thesis/img/5/quarry/all/best/*.png'))
from os import path

base_dir = '../img/5/quarry/all/best/'
size =0.19

print('\\begin{figure}')
print('\\centering')
for s in ps:
    print('\\begin{{subfigure}}[b]{{{}\\textwidth}}'.format(size))
    print('\\includegraphics[width=\linewidth]{{{}}}'.format(base_dir + path.basename(s)))
    print('\\caption{{${}$cm}}'.format(path.basename(s)[0:2]))
    print('\\end{subfigure}')
    
print('\\end{figure}')


\begin{figure}
\centering
\begin{subfigure}[b]{0.19\textwidth}
\includegraphics[width=\linewidth]{../img/5/quarry/all/best/35-patch-3d-majavi-colormap-0.png}
\caption{$35$cm}
\end{subfigure}
\begin{subfigure}[b]{0.19\textwidth}
\includegraphics[width=\linewidth]{../img/5/quarry/all/best/38-patch-3d-majavi-colormap-1.png}
\caption{$38$cm}
\end{subfigure}
\begin{subfigure}[b]{0.19\textwidth}
\includegraphics[width=\linewidth]{../img/5/quarry/all/best/44-patch-3d-majavi-colormap-2.png}
\caption{$44$cm}
\end{subfigure}
\begin{subfigure}[b]{0.19\textwidth}
\includegraphics[width=\linewidth]{../img/5/quarry/all/best/45-patch-3d-majavi-colormap-3.png}
\caption{$45$cm}
\end{subfigure}
\begin{subfigure}[b]{0.19\textwidth}
\includegraphics[width=\linewidth]{../img/5/quarry/all/best/49-patch-3d-majavi-colormap-4.png}
\caption{$49$cm}
\end{subfigure}
\begin{subfigure}[b]{0.19\textwidth}
\includegraphics[width=\linewidth]{../img/5/quarry/all/best/51-patch-3d-majavi-colormap-5.png}
\caption{$51$cm}


In [9]:
import glob

def make_figure_from_paths(ps, base_dir):
#     ps.reverse()
    from os import path

    base_dir = base_dir
    size = 0.96 / 6

    print('\\begin{figure}')
    print('\\centering')
    for s in ps:
        print('\\begin{{subfigure}}[b]{{{:.3f}\\textwidth}}'.format(size))
        print('\\includegraphics[width=\linewidth]{{{}}}'.format(base_dir + path.basename(s)))
        print('\\end{subfigure}')

    print('\\end{figure}')


# ps = sorted(glob.glob('/home/francesco/Documents/Master-Thesis/papers/Thesis/img/5/custom_patches/walls_front/all/*.png'))
# make_figure_from_paths(ps[::5])

In [11]:
ps = sorted(glob.glob('/home/francesco/Documents/Master-Thesis/papers/Thesis/img/5/custom_patches/wall_under/all/*.png'))
make_figure_from_paths(ps[::5], '/home/francesco/Documents/Master-Thesis/papers/Thesis/img/5/custom_patches/wall_under/all/')

\begin{figure}
\centering
\begin{subfigure}[b]{0.160\textwidth}
\includegraphics[width=\linewidth]{/home/francesco/Documents/Master-Thesis/papers/Thesis/img/5/custom_patches/wall_under/all/00-3d.png}
\end{subfigure}
\begin{subfigure}[b]{0.160\textwidth}
\includegraphics[width=\linewidth]{/home/francesco/Documents/Master-Thesis/papers/Thesis/img/5/custom_patches/wall_under/all/05-3d.png}
\end{subfigure}
\begin{subfigure}[b]{0.160\textwidth}
\includegraphics[width=\linewidth]{/home/francesco/Documents/Master-Thesis/papers/Thesis/img/5/custom_patches/wall_under/all/10-3d.png}
\end{subfigure}
\begin{subfigure}[b]{0.160\textwidth}
\includegraphics[width=\linewidth]{/home/francesco/Documents/Master-Thesis/papers/Thesis/img/5/custom_patches/wall_under/all/15-3d.png}
\end{subfigure}
\begin{subfigure}[b]{0.160\textwidth}
\includegraphics[width=\linewidth]{/home/francesco/Documents/Master-Thesis/papers/Thesis/img/5/custom_patches/wall_under/all/20-3d.png}
\end{subfigure}
\begin{subfigure}[b]{0.1